### Import Libraries

In [ ]:
# Import the required packages and libraries
import datetime
import os
from dotenv import load_dotenv 
from pathlib import Path
import nest_asyncio

nest_asyncio.apply()

### Import Scripts

In [2]:
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

### Import Envrionment Variables

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [4]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [5]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

### Download data from modelindex api

In [6]:
# Connecting to ModelIndex APIs 
model_data = ModelIndex(url=model_index_url, auth_client=auth_client, session=auth_client.session)

In [ ]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

In [ ]:
# Types of Objects
object_types_json = model_data.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model_data.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
# To get the objects of a type
sites_json = model_data.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# Selecting the single site
site_id = sites.list_of_ids()[0]
site_id

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model_data.get_object_descendants(
    "StringSetType", [site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

In [16]:
# All inverter data for the site
inverter_json = model_data.get_object_descendants(
    "InverterType", [site_id], "PV_Assets"
)
inverters = AnalyticsHelper(inverter_json)

In [ ]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model_data.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

### Download data from the opc ua api

In [18]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initating the OPC UA API with a fixed namespace list
opc_data = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list, auth_client=auth_client)

In [ ]:
# Live value data of trackers
live_value = opc_data.get_values(
    trackers.variables_as_list(["AngleMeasured"])
)
live_value

In [ ]:
# 1 day aggregated historical data
one_day_historical_data = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=string_sets_for_first_park.variables_as_list(["DCPower"])
)
one_day_historical_data

In [ ]:
# 1 day raw historical data
one_day_raw_historical_data = opc_data.get_historical_raw_values(
    start_time = datetime.datetime(2024, 7, 13, 00, 00),
    end_time = datetime.datetime(2024, 7, 13, 23, 59),
    variable_list=string_sets_for_first_park.variables_as_list(["DCPower"])
)
one_day_raw_historical_data